## Training MobileNetV2

In [1]:
import torch
import torchvision.models as models
import torch.nn as nn

### https://drive.google.com/file/d/10EPoE4EuFQ6Sq8VZTQmboIiQLdk1w_gl/view?usp=sharing

In [2]:
#!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=10EPoE4EuFQ6Sq8VZTQmboIiQLdk1w_gl' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=10EPoE4EuFQ6Sq8VZTQmboIiQLdk1w_gl" -O drone-dataset.zip

In [3]:
# !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1o8j2VFM1F7WLe-xmDyb6iUu8NGA8hjUy' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1o8j2VFM1F7WLe-xmDyb6iUu8NGA8hjUy" -O drone-dataset-rao.zip

In [4]:
# import numpy as np
# import cv2
# import io
# from torch.utils.data import DataLoader, Dataset
# from torchvision.transforms import ToTensor
# import zipfile

# class ZipDataset(Dataset):
#     def __init__(self, root_path, cache_into_memory=False):
#         if cache_into_memory:
#             f = open(root_path, 'rb')
#             self.zip_content = f.read()
#             f.close()
#             self.zip_file = zipfile.ZipFile(io.BytesIO(self.zip_content), 'r')
#         else:
#             self.zip_file = zipfile.ZipFile(root_path, 'r')
#         self.name_list = list(filter(lambda x: x[-4:] == '.jpg', self.zip_file.namelist()))
#         self.to_tensor = ToTensor()

#     def __getitem__(self, key):
#         buf = self.zip_file.read(name=self.name_list[key])
#         try:
#             img = self.to_tensor(cv2.resize(cv2.imdecode(np.fromstring(buf, dtype=np.uint8), cv2.IMREAD_COLOR), (224, 224), interpolation = cv2.INTER_AREA))
#             return img
#         except:
#             print("Error occured while converting this image")
        

#     def __len__(self):
#         return len(self.name_list)

# if __name__ == '__main__':
#     dataset = ZipDataset('drone-dataset.zip', cache_into_memory=True)
#     dataloader = DataLoader(dataset, batch_size=8, num_workers=0)
#     for batch_idx, sample in enumerate(dataloader):
#         print(batch_idx, sample.size())

In [5]:
import torchvision
import torch
import torchvision.transforms as transforms

def load_dataset():
    train_data_path = './drone-dataset/train'
    test_data_path = './drone-dataset/test'
    train_dataset = torchvision.datasets.ImageFolder(
        root=train_data_path,
        transform = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.Grayscale(num_output_channels=3),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    )
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=16,
        num_workers=0,
        shuffle=True
    )
    
    test_dataset = torchvision.datasets.ImageFolder(
        root=test_data_path,
        transform = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    )
    test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=16,
        num_workers=0,
        shuffle=True
    )
    return train_loader, test_loader

train_loader, test_loader = load_dataset()
# for batch_idx, (data, target) in enumerate(train_loader):
#     print(batch_idx, data.size())

In [6]:
# for batch_idx, (data, target) in enumerate(test_loader):
#     print(batch_idx, data.size())

In [7]:
from model import DroneMobilenetV2
from torchsummary import summary
from traintest import train
from traintest import test



device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

drone_mobilenet_v2 = DroneMobilenetV2()
summary(drone_mobilenet_v2.to(device), input_size=(3, 224, 224))

Using cache found in /home/verito/.cache/torch/hub/pytorch_vision_v0.6.0


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 112, 112]             864
       BatchNorm2d-2         [-1, 32, 112, 112]              64
             ReLU6-3         [-1, 32, 112, 112]               0
            Conv2d-4         [-1, 32, 112, 112]             288
       BatchNorm2d-5         [-1, 32, 112, 112]              64
             ReLU6-6         [-1, 32, 112, 112]               0
            Conv2d-7         [-1, 16, 112, 112]             512
       BatchNorm2d-8         [-1, 16, 112, 112]              32
  InvertedResidual-9         [-1, 16, 112, 112]               0
           Conv2d-10         [-1, 96, 112, 112]           1,536
      BatchNorm2d-11         [-1, 96, 112, 112]             192
            ReLU6-12         [-1, 96, 112, 112]               0
           Conv2d-13           [-1, 96, 56, 56]             864
      BatchNorm2d-14           [-1, 96,

/home/verito/Documents/EVA4P2/session2/Drone-Classification_End2End/model.py:19: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(out)


In [8]:
#drone_mobilenet_v2 = torch.load('drone_mobilenet_v2_2.pt')
drone_mobilenet_v2 = drone_mobilenet_v2.to(device)

In [9]:
drone_mobilenet_v2.freeze()

In [ ]:
for epoch in range(5):
    train.train(drone_mobilenet_v2, device, train_loader, epoch)
    test.test(drone_mobilenet_v2, device, test_loader)
    torch.save(drone_mobilenet_v2, 'drone_mobilenet_v2_3.pt')
    print("*********** Epoch {} ended *************".format(epoch))

Loss=0.3200063109397888 Accuracy=88.81:  93%|█████████▎| 654/707 [03:15<00:24,  2.13it/s]  

In [ ]:
from tqdm import tqdm

